In [ ]:
from palmerpenguins import penguins
from palmerpenguins import load_penguins
from pandas import get_dummies
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing

In [ ]:
df = penguins.load_penguins().dropna()

df.head(3)

In [ ]:
X = get_dummies(df[['bill_length_mm', 'species', 'sex']], drop_first = True)
y = df['body_mass_g']

model = LinearRegression().fit(X, y)

In [ ]:
print(f"R^2 {model.score(X,y)}")
print(f"Intercept {model.intercept_}")
print(f"Columns {X.columns}")
print(f"Coefficients {model.coef_}")


## END OF LAB 1 MODEL CREATION

In [ ]:
## LAB 2
from vetiver import VetiverModel, vetiver_pin_write, VetiverAPI
from pins import board_folder

# Create a VetiverModel instance
v = VetiverModel(model, model_name="penguin_weight_predictor", prototype_data=df[['bill_length_mm', 'species', 'sex']])

# Set up a board and write the model to it
model_board = board_folder("data/model", allow_pickle_read=True)
vetiver_pin_write(model_board, v)

# Create and run the API on a different port if 8080 is in use
app = VetiverAPI(v, check_prototype=True)
try:
    app.run(port=8082)  # Change port number to avoid conflict
except Exception as e:
    print(f"Failed to start server: {e}")

## LAB 3
# Connect to DuckDB and perform operations
con = duckdb.connect('my-db.duckdb')
con.execute("CREATE TABLE IF NOT EXISTS penguins AS SELECT * FROM load_penguins()")
df = con.execute("SELECT * FROM penguins").fetchdf().dropna()
con.close()